# Importación de módulos

In [1]:
import asyncio
import os
import spacy
import sys

from aiofile import AIOFile
from aiohttp import ClientSession
from bs4 import Comment

from src.data_obtaining.pdf_processing import convert_pdf_to_txt
from src.data_obtaining.webpage_interaction import download_document_and_save
from src.data_obtaining.webpage_interaction import get_webpage_as_bs
from src.processing.constants import BASE_DIRECTORY
from src.preparation.models.obtained_text import ObtainedText
from src.preparation.data_access.obtained_text_da import ObtainedTextDA

/home/hans/Documentos/Tesis_Chatbot/.env/lib/python3.8/site-packages/sqlalchemy/orm/util.py:104: SAWarning: The 'delete-orphan' cascade option requires 'delete'.
  util.warn(


# Constantes

In [2]:
'''URLS = ['http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=20',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=17',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=18']'''

'''URLS = ['http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=11&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=17&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=11&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=20&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=11&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=18&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=13&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=17&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=13&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=18&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=9&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=17&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=9&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=18&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=9&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=20&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=5&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=20&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=18&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=20&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=9&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=17&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=9&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=18&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=9&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=20&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=4&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=18&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=7&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=20&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=4&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=6&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=13&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=6&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10']'''
        

URLS = ['http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=4&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=6&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=2&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10',
        'http://www.perueduca.pe/materiales-educativos?p_p_id=ResourcesPublicPE_WAR_ResourcesPublicPEportlet&p_p_lifecycle=0&p_p_state=normal&p_p_mode=view&p_p_col_id=column-1&p_p_col_count=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_jspPage=%2Farea.jsp&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_q=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_areaId=13&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_tiporec=6&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_gradoId=3&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_filter=&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_inicio=1&_ResourcesPublicPE_WAR_ResourcesPublicPEportlet_fin=10']

In [3]:
LEVELS = {
    'Educación Primaria': '1',
    'Educación Secundaria': '2'
}

In [4]:
CATEGORIES = {
    'Historia, Geografía y Economía': 'Historia, Geografía y Economía',
    'CTA': 'CTA',
    'Ciencia y tecnología': 'CTA',
    'Ciencia y Ambiente': 'CTA',
    'Arte': 'Historia, Geografía y Economía',
    'Comunicación': 'Comunicación',
    'Personal Social': 'Historia, Geografía y Economía',
    'Educación Religiosa': 'Historia, Geografía y Economía',
}

In [5]:
PROJECT_PATH = '/home/hans/Documentos/Tesis_Chatbot'

# Funciones extra

In [6]:
def is_accepted_document(document_language: str, document_category: str, document_name: str) -> bool:
    """
    This function checks if the document to download has the correct language and belongs to the correct category
    
    Parameters:
    document_language (str): Language the document was written in.
    document_category (str): Teaching category the document belongs to.
    document_name (str): The name of the document
    
    Returns:
    Boolean: If the document is accepted or not
    """
    accepted_language = 'ES (Español)'
    accepted_areas = ['Formación Ciudadana y Cívica', 
                  'Comunicación', 
                  'CTA',
                  'Ciencia y tecnología',
                  'Ciencia y Ambiente',
                  'Personal Social',
                  'Historia, Geografía y Economía',
                  'Ciencias sociales']
    texts_to_omit = ['unuchamanta-kawsaq.pdf',
                     'sankenapatotantsi-ashaninka.pdf',
                     'phuyu-raqeaq-atuq.pdf',
                     'arojeitake-ashitarori-impoiji-amabentakotirori-aipatsite.pdf',
                     'kantilarya-q-uchuna.pdf',
                     'qamaqimpi-wallatampi.pdf',
                     'siku-phusa.pdf',
                     'yana-t-urumanta-t-iqirunacha.pdf',
                     'zampona-siku.pdf',
                     'papa-mamanchikmanta.pdf',
                     'santurantikuy.pdf',
                     'awila-mikayla.pdf',
                     'Ruta_Shawi_Inicial.pdf',
                     'Ruta_aimara_inicial.pdf',
                     'Ruta_ashaninka_Inicial.pdf',
                     'Ruta_castellano_amazonico_inicial.pdf',
                     'Ruta_Shawi_ComMat_IIIciclo.pdf',
                     'Ruta_Shawi_ComMat_IVciclo.pdf',
                     'Ruta_Shawi_ComMat_Vciclo.pdf',
                     'Ruta_Shipibo_ComMat_IVciclo.pdf'
                     'Ruta_Shipibo_ComMat_Vciclo.pdf',
                     'Ruta_aimara_ComMat_III_ciclo.pdf',
                     'Ruta_ashaninka_ComMat_IV_ciclo.pdf',
                     'Ruta_ashaninka_ComMat_Vciclo.pdf',
                     'Ruta_castellano_amazonico_IVciclo.pdf''Ruta_quechuaCollao_ComMat_IVciclo.pdf',
                    'Ruta_quechuaCollao_ComMat_Vciclo.pdf',
                    'cartilla_ashaninka.pdf',
                    'cartilla_matsigenka.pdf',
                    'guia_alfabeto_cashinahua.pdf',
                    'guia_alfabeto_ese_eja.pdf',
                    'guia_alfabeto_jaqaru.pdf',
                    'guia_alfabeto_kichwa.pdf',
                    'guia_alfabeto_matsigenka.pdf',
                    'guia_alfabeto_murui_muinani.pdf',
                    'guia_alfabeto_secoya.pdf',
                    'guia_alfabeto_sharanahua.pdf',
                    'guia_alfabeto_wampis.pdf',
                    'guia_alfabeto_yine.pdf',
                    'manual_ashaninka.pdf',
                    'manual_awajun.pdf',
                    'manual_ese_eja.pdf',
                    'manual_harakbut.pdf',
                    'manual_jaqaru.pdf',
                    'manual_kakataibo.pdf',
                    'manual_quechua_collao.pdf',
                    'manual_shawi.pdf',
                    'manual_shipibo.pdf',
                    'manual_yanesha.pdf',
                    'manual_yine.pdf',
                    'pegantatsiri-ogotaige-omagaro-2.pdf',
                    'Text Ruta_quechuaCollao_ComMat_IVciclo.pdf',
                    'Text Ruta_quechuaCollao_ComMat_Vciclo.pdf',
                    'mi-cuaderno-autoaprendizaje-ps-4']
    print(document_language == accepted_language, document_category in accepted_areas, not document_name in texts_to_omit, document_name)
    return document_language == accepted_language and document_category in accepted_areas and not document_name in texts_to_omit

In [7]:
async def obtain_and_save_document(session: ClientSession, url: str) -> None:
    """
    This function looks for the link to download a document in "perueduca.com"

    Parameters:
    session (ClientSession): The async session used to obtain the webpage
    url (str): The url of the webpage to convert

    Returns:
    None
    """
    try:
        soup = await get_webpage_as_bs(session, url)
        document_information_table = soup.find('div', id='ficha-catalogo').find_all('tr')
        document_language = document_information_table[7].find('td').text.strip()
        document_category = document_information_table[3].find('td').text.strip()
        download_link = soup.find('a', id='hf_iframe')['href']
        document_name = download_link.split('/')[-1]
        if (is_accepted_document(document_language, document_category, document_name)):
            document_level = LEVELS[document_information_table[1].find('td').text.strip()]
            document_folder_category = CATEGORIES[document_category]
            storage_directory = BASE_DIRECTORY + f'/data/raw/pdf/{document_level}/{document_folder_category}/{document_name}'
            if '.pdf' in download_link:
                await download_document_and_save(session, download_link, storage_directory)
                print(f'Document {document_name} downloaded successfully')
    except Exception as e:
        raise e

In [8]:
async def obtain_texts_from_perueduca_webpage(session: ClientSession, url: str) -> None:
    """
    This function traverses through an entire web page of peru educa, searching for documents to download.
    It's a recursive function.
    
    This function begins with obtaining the webpage as a beautiful soup object, then, it proceeds to find
    all the available document sections in this page so it can download them. After that, it goes to the
    next page and does the same process again.

    Parameters:
    session (ClientSession): The async session used to obtain the webpage
    url (str): The url of the webpage to convert

    Returns:
    None
    """
    try:
        soup = await get_webpage_as_bs(session, url)
        texts_information = soup.find_all('div', class_='box-result')
        if (len(texts_information) == 0): # No more texts in this page. Extraction process finished
            return
        else: # More texts still available
            for text in texts_information:
                text_link = text.find('a')['href']
                await obtain_and_save_document(session, text_link)
            # There are still more pages to look documents into
            next_page_button = soup.find('li', class_='next')
            next_page_url = next_page_button.find('a')['href']
            await obtain_texts_from_perueduca_webpage(session, next_page_url)
    except Exception as e:
        raise e


In [9]:
async def convert_pdfs_to_txt_files() -> None:
    """
    This function scans all the files we just downloaded and converts them to a .txt format for easier processing
    """
    levels = ['1', '2']
    categories = ['Historia, Geografía y Economía',
                    'CTA',
                    'Comunicación']
    for level in levels:
        for category in categories:
            documents = [file_name for file_name in os.listdir(BASE_DIRECTORY + f'/data/raw/pdf/{level}/{category}') if '.pdf' in file_name]
            documents.sort()
            for document in documents:
                try:
                    pdf_path = f'{BASE_DIRECTORY}/data/raw/pdf/{level}/{category}/{document}'
                    save_dir = f'{BASE_DIRECTORY}/data/raw/txt/{level}/{category}/{document}'.replace('.pdf', '.txt')
                    await convert_pdf_to_txt(pdf_path, save_dir)
                    print(f'Text {document} converted from pdf to txt')
                except Exception as e:
                    print(str(e))
                    continue

In [10]:
async def save_text_files_to_database() -> None:
    '''
    This will save all texts to a sqlite database.
    '''         
    obtained_text_da = ObtainedTextDA()
    levels = ['1', '2']
    categories = ['Historia, Geografía y Economía',
                    'CTA',
                    'Comunicación']
    obtained_texts = obtained_text_da.select_all()
    try:
        for level in levels:
            for category in categories:
                documents = [file_name for file_name in os.listdir(BASE_DIRECTORY + f'/data/raw/txt/{level}/{category}') if '.txt' in file_name]
                documents.sort()
                for document in documents:
                    already_found = [ot for ot in obtained_texts if ot.filename == document]
                    if len(already_found) != 0:
                        print(f'{document} has already been processed.')
                    else:
                        async with AIOFile(f'{BASE_DIRECTORY}/data/raw/txt/{level}/{category}/{document}', 'r') as f:
                            text = await f.read()
                            if len(text) > 0:
                                obtained_text = ObtainedText(text=text, grade=level, filename=document, category=category)
                                obtained_text_da.insert(obtained_text)
                                print(f'Text {document} saved to dabatase.')
                            else:
                                print(f'Text {document} was empty.')
    except Exception as e:
        print(str(e))

In [11]:
async def main() -> None:
    """
    The main function obtains the documents as pdfs and then converts them to text file format
    """
    '''try:
        session = ClientSession()
        for url in URLS:
            await obtain_texts_from_perueduca_webpage(session, url)
        await session.close()
    except Exception as e:
        await session.close()'''
    await convert_pdfs_to_txt_files()
    await save_text_files_to_database()

In [12]:
await main()

2020-09-17 14:03:53,191 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar to /tmp/tika-server.jar.
2020-09-17 14:04:13,009 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.24/tika-server-1.24.jar.md5 to /tmp/tika-server.jar.md5.
2020-09-17 14:04:14,007 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Text atahualpa.pdf converted from pdf to txt
Text avelino_caceres.pdf converted from pdf to txt
Text cartilla_fenomeno_nino.pdf converted from pdf to txt
Text cartilla_inundaciones.pdf converted from pdf to txt
Text enrique_meiggs.pdf converted from pdf to txt
Text epopeya_arica.pdf converted from pdf to txt
Text francisco_bolognesi.pdf converted from pdf to txt
Text francisco_orellana.pdf converted from pdf to txt
Text francisco_toledo.pdf converted from pdf to txt
Text guaman_poma.pdf converted from pdf to txt
Text hipolito_unanue.pdf converted from pdf to txt
Text ines_huaylas.pdf converted from pdf to txt
Text jorge_basadre.pdf converted from pdf to txt
Text jorge_chavez.pdf converted from pdf to txt
Text jose_olaya.pdf converted from pdf to txt
Text jsanchez_carrion.pdf converted from pdf to txt
Text juanita.pdf converted from pdf to txt
Text leoncio_prado.pdf converted from pdf to txt
Text manco_inca.pdf converted from pdf to txt
Text mateo_pumacahua.pdf converted from pdf to txt

Text 20_leguas_viaje_submarino.pdf converted from pdf to txt
Text abel_sanchez.pdf converted from pdf to txt
Text amistad_funesta.pdf converted from pdf to txt
Text antologia-literaria-1.pdf converted from pdf to txt
Text antologia-literaria-2.pdf converted from pdf to txt
Text antologia-literaria-4.pdf converted from pdf to txt
Text antologia-literaria-5.pdf converted from pdf to txt
Text antologia_leer_escribir.pdf converted from pdf to txt
Text arguedas.pdf converted from pdf to txt
Text aventuras_robinson_crusoe.pdf converted from pdf to txt
Text aventuras_tom_sawyer.pdf converted from pdf to txt
Text bella_bestia.pdf converted from pdf to txt
Text bibliotecas_escolares.pdf converted from pdf to txt
Text cartas_sor_juana.pdf converted from pdf to txt
Text castillo.pdf converted from pdf to txt
Text cesar_vallejo.pdf converted from pdf to txt
Text charles_dickens.pdf converted from pdf to txt
Text clorinda_matto.pdf converted from pdf to txt
Text comunicacion-secundaria-rural-cuader

Text mi-cuaderno-autoaprendizaje-com-6.txt saved to dabatase.
Text proyecto-aprendizaje-portafolio-inicial.txt saved to dabatase.
Text situaciones-comunicativas-comunicacion-texto-1-inicial.txt saved to dabatase.
Text situaciones-comunicativas-texto-2-inicial.txt saved to dabatase.
Text situaciones-para-aprender-construir-portafolio-2-intermedio.txt saved to dabatase.
Text situaciones-para-aprender-construir-portafolio-intermedio-1.txt saved to dabatase.
Text situaciones-para-aprender-construir-texto-2-intermedio.txt saved to dabatase.
Text situaciones-para-aprender-construir-texto-3-intermedio.txt saved to dabatase.
Text situaciones-para-aprender-construir-texto-intermedio-1.txt saved to dabatase.
Text Sin título 1.txt saved to dabatase.
Text augusto_leguia.txt saved to dabatase.
Text canal_panama.txt saved to dabatase.
Text carta_jamaica.txt saved to dabatase.
Text cuenca_rio_santa.txt saved to dabatase.
Text expansion_roma.txt saved to dabatase.
Text fernando_belaunde.txt saved to d